In [3]:
import numpy as np

class AntColonyOptimization:
    def __init__(self, n_cities, alpha, beta, rho, Q):
        self.n_cities = n_cities
        self.alpha = alpha  # pheromone importance
        self.beta = beta    # distance importance
        self.rho = rho      # pheromone evaporation rate
        self.Q = Q          # pheromone deposit factor
        
        # Initialize pheromone and distance matrices
        self.tau = np.ones((n_cities, n_cities))
        self.eta = 1 / np.random.rand(n_cities, n_cities)  # Example: inverse of distance
        np.fill_diagonal(self.eta, 0)  # Set diagonal to 0
        
    def calculate_probabilities(self, ant_k, city):
        allowed = list(set(range(self.n_cities)) - set(ant_k))
        probabilities = np.zeros(self.n_cities)
        
        for y in allowed:
            numerator = (self.tau[city][y] ** self.alpha) * (self.eta[city][y] ** self.beta)
            denominator = sum((self.tau[city][z] ** self.alpha) * (self.eta[city][z] ** self.beta) for z in allowed)
            probabilities[y] = numerator / denominator
        
        return probabilities
    
    def update_pheromone(self, delta_tau):
        self.tau = (1 - self.rho) * self.tau + delta_tau
    
    def calculate_delta_tau(self, ant_path, path_length):
        delta_tau = np.zeros((self.n_cities, self.n_cities))
        for i in range(len(ant_path) - 1):
            x, y = ant_path[i], ant_path[i+1]
            delta_tau[x][y] = self.Q / path_length
        return delta_tau
    
    def run(self, n_ants, n_iterations):
        best_path = None
        best_path_length = float('inf')
        
        for _ in range(n_iterations):
            all_paths = []
            all_path_lengths = []
            
            for _ in range(n_ants):
                ant_path = self.construct_solution()
                path_length = self.calculate_path_length(ant_path)
                all_paths.append(ant_path)
                all_path_lengths.append(path_length)
                
                if path_length < best_path_length:
                    best_path = ant_path
                    best_path_length = path_length
            
            # Update pheromone
            delta_tau = sum(self.calculate_delta_tau(path, length) 
                            for path, length in zip(all_paths, all_path_lengths))
            self.update_pheromone(delta_tau)
        
        return best_path, best_path_length
    
    def construct_solution(self):
        ant_path = [np.random.randint(self.n_cities)]
        while len(ant_path) < self.n_cities:
            probabilities = self.calculate_probabilities(ant_path, ant_path[-1])
            next_city = np.random.choice(self.n_cities, p=probabilities)
            ant_path.append(next_city)
        return ant_path
    
    def calculate_path_length(self, path):
        return sum(self.eta[path[i]][path[i+1]] for i in range(len(path)-1))

# Example usage
aco = AntColonyOptimization(n_cities=10, alpha=1, beta=2, rho=0.5, Q=100)
best_path, best_length = aco.run(n_ants=20, n_iterations=100)
print(f"Best path: {best_path}")
print(f"Best path length: {best_length}")

Best path: [1, 3, 6, 4, 0, 8, 2, 9, 5, 7]
Best path length: 42.75314428217791


In [21]:
import numpy as np
from collections import defaultdict

class AntColonyOptimizationO1:
    def __init__(self, n_cities, alpha, beta, rho, Q):
        self.n_cities = n_cities
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.Q = Q
        
        # Initialize pheromone and distance matrices
        self.tau = np.ones((n_cities, n_cities))
        self.eta = 1 / np.random.rand(n_cities, n_cities)
        np.fill_diagonal(self.eta, 0)
        
        # Precompute tau^alpha * eta^beta for O(1) probability calculations
        self.pheromone_times_visibility = (self.tau ** alpha) * (self.eta ** beta)
        
        # Use a dict for O(1) lookup of allowed cities
        self.allowed_cities = defaultdict(set)
        for i in range(n_cities):
            self.allowed_cities[i] = set(range(n_cities)) - {i}
    
    def select_next_city(self, current_city, visited):
        allowed = self.allowed_cities[current_city] - visited
        if not allowed:
            return None
        
        # Calculate probabilities using precomputed values
        probabilities = [self.pheromone_times_visibility[current_city][city] for city in allowed]
        total = sum(probabilities)
        probabilities = [p / total for p in probabilities]
        
        return np.random.choice(list(allowed), p=probabilities)
    
    def update_pheromone(self, delta_tau):
        self.tau = (1 - self.rho) * self.tau + delta_tau
        # Update precomputed values
        self.pheromone_times_visibility = (self.tau ** self.alpha) * (self.eta ** self.beta)
    
    def construct_solution(self):
        start_city = np.random.randint(self.n_cities)
        path = [start_city]
        visited = {start_city}
        
        while len(path) < self.n_cities:
            next_city = self.select_next_city(path[-1], visited)
            if next_city is None:
                break
            path.append(next_city)
            visited.add(next_city)
        
        return path
    
    def calculate_path_length(self, path):
        return sum(self.eta[path[i]][path[i+1]] for i in range(len(path)-1))
    
    def run(self, n_ants, n_iterations):
        best_path = None
        best_path_length = float('inf')
        
        for _ in range(n_iterations):
            all_paths = [self.construct_solution() for _ in range(n_ants)]
            all_path_lengths = [self.calculate_path_length(path) for path in all_paths]
            
            # Update best path
            min_length = min(all_path_lengths)
            if min_length < best_path_length:
                best_path = all_paths[all_path_lengths.index(min_length)]
                best_path_length = min_length
            
            # Update pheromone
            delta_tau = np.zeros((self.n_cities, self.n_cities))
            for path, length in zip(all_paths, all_path_lengths):
                for i in range(len(path) - 1):
                    x, y = path[i], path[i+1]
                    delta_tau[x][y] += self.Q / length
            
            self.update_pheromone(delta_tau)
        return best_path, best_path_length

# Example usage
aco = AntColonyOptimizationO1(n_cities=10, alpha=1, beta=2, rho=0.5, Q=100)
best_path, best_length = aco.run(n_ants=20, n_iterations=100)
print(f"Best path: {best_path}")
print(f"Best path length: {best_length}")

Best path: [9, 0, 7, 1, 2, 4, 5, 6, 3, 8]
Best path length: 30.719260603984235
